In [ ]:
import os
import shutil
import subprocess

from tqdm.notebook import tqdm

#list all input files
input_file_dir = "./lpla_trimmed"

input_files = []
skip_files = []
for root, dirs, files in os.walk(input_file_dir, topdown=False):
    for name in files:
        fp = os.path.join(root, name)
        print(fp)

        if fp.endswith(".dat"):
            input_files.append(fp)
        else:
            skip_files.append(fp)

In [ ]:
import os
import shutil
import subprocess

import pandas as pd
from tqdm.notebook import tqdm

#pick models to use for ffmaker
pdb_set = {
    "lpla_closed_mono_CHAI1.pdb",
    "lpla_open_mono_3A7R_chainA.pdb",
    "lpla_open_dimer_3A7R.pdb",
    "lpla_closed_tetramer_AF3.pdb",
    "lpla_closed_dimer_AF3.pdb",
}

output_dir = "./oligomer_outputs/"
os.makedirs(output_dir, exist_ok=True)

smax_cutoff = 0.4
remake_ff = False

#run ffmaker if ff file doesnt exist
set_output_dir = os.path.join(output_dir, pdb_set)
os.makedirs(set_output_dir, exist_ok=True)
out_ff = f"lpla_ff_{pdb_set}_{smax_cutoff}.dat"
if not os.path.isfile(out_ff) or remake_ff:
    pdbs = " ".join(pdb_sets[pdb_set])
    print(pdbs)
    subprocess.check_output(
        f"ffmaker {pdbs} -smin 0.0 -smax {smax_cutoff} -ns 501 -o {out_ff}",
        shell=True,
    )
else:
    print(f"FF already exists ({out_ff}), skipping...")

#iterate through listed files
for file in tqdm(input_files):
    file_prefix = os.path.splitext(os.path.basename(file))[0]
    outfile = file_prefix + f"_{smax_cutoff}.log"

    #generate oligomer output if it doesn't already exist
    outputs_exist = (outfile) in os.listdir(set_output_dir)
    if not outputs_exist:
        subprocess.check_output(
            f"oligomer {file} -ff {out_ff} -smin 0.0 -smax {smax_cutoff} -out {set_output_dir}/{outfile}",
            shell=True,
        )
    else:
        print(f"Skipping {file_prefix}, analysis already completed...")

    #copy fit file to output directory
    fname = file_prefix + ".fit"
    out_fname = file_prefix + f"_{smax_cutoff}" + ".fit"

    shutil.move(
        os.path.join("./", fname), os.path.join("./", set_output_dir, out_fname)
    )